TODO
- [ ] Write methodology  
- [ ] Remove irrelevant data columns in the preprocessing  
- [ ] Salary above grant and living wage  
- [ ] Number of places available per area + average and median salary by geography  
- [ ] Number of visa available places over 3 years  
- [ ] Number of split placements available over 3 years  
- [ ] Salary by tye of placement  
- [ ] Hourly wage graph with minimum wage (for 21+) and living wage  
- [ ] Hours worked per week (expectation to work on the weekends and lates in hospitals)  



- https://survey.stackoverflow.co/2024/
- https://www.gov.uk/national-minimum-wage-rates

- Make online database using openai API to summarise the programme description into quick bullet points with filter function e.g. geographical

In [16]:
library(dplyr)
library(ggplot2)
library(tidyr)
library(openai)
library(purrr)

# Load the data
OD2526 <- read.csv("oriel-training-year-2526.csv")

OD2425 <- read.csv("oriel-training-year-2425.csv")

OD2324 <- read.csv("oriel-training-year-2324.csv")

OD2223 <- read.csv("oriel-training-year-2223.csv")

In [35]:
# Check for colnames for each dataset
colnames2526 <- colnames(OD2526)
colnames2425 <- colnames(OD2425)
colnames2324 <- colnames(OD2324)
colnames2223 <- colnames(OD2223)

equal_2526_2425 <- all.equal(colnames2526, colnames2425)
equal_2526_2324 <- all.equal(colnames2526, colnames2324)
equal_2526_2223 <- all.equal(colnames2526, colnames2223)
equal_2425_2324 <- all.equal(colnames2425, colnames2324)
equal_2425_2223 <- all.equal(colnames2425, colnames2223)
equal_2324_2223 <- all.equal(colnames2324, colnames2223)

print(paste("Columns 2526 and 2425:", equal_2526_2425))
print(paste("Columns 2526 and 2324:", equal_2526_2324))
print(paste("Columns 2526 and 2223:", equal_2526_2223))
print(paste("Columns 2425 and 2324:", equal_2425_2324))
print(paste("Columns 2425 and 2223:", equal_2425_2223))
print(paste("Columns 2324 and 2223:", equal_2324_2223))

[1] "Columns 2526 and 2425: 14 string mismatches"


[1] "Columns 2526 and 2324: Lengths (39, 29) differ (string compare on first 29)"
[2] "Columns 2526 and 2324: 29 string mismatches"                                
[1] "Columns 2526 and 2223: Lengths (39, 29) differ (string compare on first 29)"
[2] "Columns 2526 and 2223: 29 string mismatches"                                
[1] "Columns 2425 and 2324: Lengths (39, 29) differ (string compare on first 29)"
[2] "Columns 2425 and 2324: 29 string mismatches"                                
[1] "Columns 2425 and 2223: Lengths (39, 29) differ (string compare on first 29)"
[2] "Columns 2425 and 2223: 29 string mismatches"                                
[1] "Columns 2324 and 2223: TRUE"


ERROR: [1m[33mError[39m in `setdiff()`:[22m
[1m[22m[33m![39m `x` and `y` are not compatible.
[31m✖[39m Cols in `y` but not `x`: `ICS.Name`,
  `Number.of.Placements..3.months.or.more.`, `Practice.Placement.Setting.1`,
  `Length.of.Placement.1..3.months.or.more.`, `Practice.Placement.Setting.2`,
  `Length.of.Placement.2..3.months.or.more.`, `Practice.Placement.Setting.3`,
  `Length.of.Placement.3..3.months.or.more.`, `Practice.Placement.Setting.4`,
  `Length.of.Placement.4..3.months.or.more.`, `Placement.1..SystemID`,
  `Placement.2..SystemID`, `Placement.3..SystemID`, `Placement.4..SystemID`.
[31m✖[39m Cols in `x` but not `y`: `ICB.Name`, `Number.of.Placements.Rotations`,
  `Practice.Placement.Rotation.Setting.1`,
  `Length.of.Placement.Rotation.1..weeks.`,
  `Practice.Placement.Rotation.Setting.2`,
  `Length.of.Placement.Rotation.2..weeks.`,
  `Practice.Placement.Rotation.Setting.3`,
  `Length.of.Placement.Rotation.3..weeks.`,
  `Practice.Placement.Rotation.Setting.4`,
  `L

In [ ]:
OD <- as.data.frame(OD) 

names(OD)[names(OD) == "Salary...."] <- "Salary"
names(OD)[names(OD) == "Skilled.worker.visa"] <- "Visa"
names(OD)[names(OD) == "Hours.per.Week"] <- "Working.hours"
names(OD)[names(OD) == "Practice.Placement.Rotation.Setting.1"] <- "Rotation1"
names(OD)[names(OD) == "Length.of.Placement.Rotation.1..weeks."] <- "Rotation1.duration"
names(OD)[names(OD) == "Practice.Placement.Rotation.Setting.2"] <- "Rotation2"
names(OD)[names(OD) == "Length.of.Placement.Rotation.2..weeks."] <- "Rotation2.duration"
names(OD)[names(OD) == "Practice.Placement.Rotation.Setting.3"] <- "Rotation3"
names(OD)[names(OD) == "Length.of.Placement.Rotation.3..weeks."] <- "Rotation3.duration"
names(OD)[names(OD) == "Practice.Placement.Rotation.Setting.4"] <- "Rotation4"
names(OD)[names(OD) == "Length.of.Placement.Rotation.4..weeks."] <- "Rotation4.duration"

colnames(OD)
dim(OD)  

#Classify the type of training
OD$Training.type <- ifelse(OD$Rotation1.duration >= OD$Rotation2.duration + 13, "Non-split", 
                                      ifelse(OD$Rotation1.duration == OD$Rotation2.duration, "Split", "Other"))


In [ ]:
keeps <- c("Training.type","Employer.Type", "Region", "Sector", "Programme.Title", "Visa", "Salary", "Places.Available", "Working.hours",
           "Rotation1", "Rotation1.duration", "Rotation2", "Rotation2.duration", "Rotation3", "Rotation3.duration", "Rotation4", "Rotation4.duration")
           
OD <- OD[keeps]
colnames(OD)
OD$Hourly.Wage <- OD$Salary/52/OD$Working.hours
OD$Visa <- as.factor(OD$Visa)

In [ ]:
library(ggplot2)

# Define the plot for Hours Worked per Week
hours.worked.plot <- ggplot(OD, aes(x = Working.hours)) + 
  geom_histogram(binwidth = 1) + 
  ggtitle("Histogram of Hours Worked per Week") + 
  xlab("Hours per Week") + 
  ylab("Frequency") +
  scale_x_continuous(breaks = seq(0, max(OD$Working.hours, na.rm = TRUE), by = 5))

# Display the plot
hours.worked.plot


In [ ]:
# Define the plot for Salary with meaningful intervals
salary.plot <- ggplot(OD, aes(x = Salary)) + 
  geom_histogram(binwidth = 500) + 
  ggtitle("Histogram of Salary") + 
  xlab("Salary") + 
  ylab("Frequency") +
  scale_x_continuous(breaks = seq(20000, max(OD$Salary, na.rm = TRUE), by = 2500))

# Display the plot

length(unique(OD$Salary))
unique(OD$Salary)

# Define the plot for Salary with meaningful intervals
salary.plot <- ggplot(OD, aes(x = Salary)) + 
  geom_histogram(binwidth = 1500) + 
  ggtitle("Histogram of Salary") + 
  xlab("Salary") + 
  ylab("Frequency") +
  scale_x_continuous(breaks = seq(20000, max(OD$Salary, na.rm = TRUE), by = 2500)) +
  theme_bw() +
  geom_vline(xintercept = 25600, color = "red") 

        

salary.plot



In [ ]:
community.pharmacy.average.salary <- mean(OD$Salary[OD$Employer.Type == "Primary Care"], na.rm = TRUE)
community.pharmacy.average.salary
community.min.salary <- min(OD$Salary[OD$Employer.Type == "Primary Care"], na.rm = TRUE)
community.max.salary <- max(OD$Salary[OD$Employer.Type == "Primary Care"], na.rm = TRUE)
paste0("The average salary for community pharmacy is ", community.pharmacy.average.salary, " with a minimum of ", community.min.salary, " and a maximum of ", community.max.salary)

hospital.pharmacy.average.salary <- mean(OD$Salary[OD$Employer.Type == "Hospital"], na.rm = TRUE)
hospital.pharmacy.average.salary
hospital.min.salary <- min(OD$Salary[OD$Employer.Type == "Hospital"], na.rm = TRUE)
hospital.max.salary <- max(OD$Salary[OD$Employer.Type == "Hospital"], na.rm = TRUE)
paste0("The average salary for hospital pharmacy is ", hospital.pharmacy.average.salary, " with a minimum of ", hospital.min.salary, " and a maximum of ", hospital.max.salary)

In [ ]:
num_hosp <- filter(OD, Employer.Type == "Hospital") %>% nrow()  
num_hosp

soreted_salary <- OD[order(-OD$Salary),]
# head(soreted_salary)

# 34098 might be a wrong salary

OD <- OD %>% mutate(Salary = ifelse(Salary == 34098,34089,Salary))
soreted_salary[70,]
soreted_salary[80,]

#https://faq.nhsbsa.nhs.uk/knowledgebase/article/KA-23439/en-us
#https://www.healthcareers.nhs.uk/working-health/working-nhs/nhs-pay-and-benefits/agenda-change-pay-rates


num_hosp_with_london_weighting <- filter(OD, Employer.Type == "Hospital" & Salary == 34089) %>% nrow()  
num_hosp_with_london_weighting
# This is not the total number of places available in hospitals, but the number of programmes with London weighting

In [ ]:
result <- OD %>%
    filter(Salary == 34098) 
result

In [ ]:
OD2324 <- read.csv("oriel2324.csv")
sum(OD2324$Places.Available)
ggplot(OD2324, aes(x = Salary)) + 
  geom_histogram(binwidth = 1500) + 
  ggtitle("Histogram of Salary") + 
  xlab("Salary") + 
  ylab("Frequency") +
  scale_x_continuous(breaks = seq(20000, max(OD$Salary, na.rm = TRUE), by = 2500)) +
  theme_bw() +
  geom_vline(xintercept = 18600, color = "red") 

In [ ]:
#Expand the data frame to reflect true number of places available
sample_plot <- OD %>%
    uncount(Places.Available) 

colnames(sample_plot)

ggplot(sample_plot, aes(x = Salary)) + 
  geom_histogram(binwidth = 1500) + 
  ggtitle("Histogram of Salary") + 
  xlab("Salary") + 
  ylab("Frequency") +
  theme_bw() +
  geom_vline(xintercept = 25600, color = "red")

colnames(OD)

In [ ]:
colnames(OD2324)
OD2324_expanded <- OD2324 %>%
    uncount(Places.Available) 

ggplot(OD2324_expanded, aes(x = Salary)) + 
  geom_histogram(binwidth = 1500) + 
  ggtitle("Histogram of Salary") + 
  xlab("Salary") + 
  ylab("Frequency") +
  theme_bw() +
  geom_vline(xintercept = 18600, color = "red")

count(OD2324_expanded)

In [ ]:
readRenviron("/.renviron")

In [ ]:
OPENAI_API_KEY <- Sys.getenv("open_ai_api_key")

Sys.setenv(
    OPENAI_API_KEY = OPENAI_API_KEY
)

In [ ]:
sample_programme_description = sample_OD$Programme.Description[2106]

sample_prompt = paste0("List three very brief bullet points why this programme is good and create relevant tags for sorting: ", sample_programme_description)

In [ ]:
sample_OD$Programme.Description[2106]


In [ ]:
example_gpt <- create_chat_completion(
    model = "gpt-3.5-turbo",
    messages = list(
        list(
            "role" = "user",
            "content" = sample_prompt
        )
    )
)
example_gpt

In [ ]:
example_gpt$choices$message.content